In [1]:
import gym
import gym_sokoban
import time
import datetime as dt
import matplotlib.pyplot as plt
import math
from collections import deque
import random
import numpy as np
import pickle
import sys
import tqdm
import os
from collections import namedtuple,defaultdict,deque 

In [2]:
chapter = 0
level = 1


env_name = 'Sokoban-v1'
env = gym.make(env_name)
ACTION_LOOKUP = env.unwrapped.get_action_lookup()
env.unwrapped.set_level(chapter,level)
env.seed(0)
env.reset()
print("Created environment: {}".format(env_name))


#create Results/Chapter [chapter]/Level [level] folder
if not os.path.exists('Results/Chapter '+str(chapter)+'/Level '+str(level)):
    os.makedirs('Results/Chapter '+str(chapter)+'/Level '+str(level))

Created environment: Sokoban-v1


In [3]:
env.render()

333300333
333000333
333000333
333000333
333000333
333000333
333000333
333000333
333000333
333000333
333000333
333003333
333300333
333000333
333000333
333000333
333000333
333000333
333000333
333000333
333000333
333000333
333000333
333003333


True

In [4]:
while True:
    time.sleep(1/180)
    env.render()

KeyboardInterrupt: 

In [3]:
def greedy_policy(V,s):
    #.9 prob of greedy action
    #.1 prob of random action

    if s not in V:
        V[s] = np.zeros(env.action_space.n)
    r_choice = .4
    if np.random.random() < r_choice:
        return np.random.choice(np.arange(env.action_space.n))
    else:
        max_val = np.max(V[s])
         #find all actions that have the max value and choose one at random
        max_actions = np.argwhere(V[s] == max_val).flatten()
        return np.random.choice(max_actions)

In [4]:
EVERY_VISIT_MC = False
EPISODES = 1000

V = {}
total_returns = {}
N = {}
for episode in tqdm.tqdm(range(EPISODES)):
    
    visited = []
    env.reset()
    state = env.unwrapped.serialize_state()
    done = False
    for t in range(40):
        action_time = time.time()
        if done:
            break
        action = greedy_policy(V,state)
        next_state, reward, done, info = env.step(action)
        next_state = env.unwrapped.serialize_state()
        if EVERY_VISIT_MC or (not EVERY_VISIT_MC and state not in visited):
            if not EVERY_VISIT_MC:
                visited.append(state)
            if state not in total_returns:
                total_returns[state] = np.zeros(env.action_space.n)
            for _state in total_returns:
                total_returns[_state][action] += reward
            if state not in N:
                N[state] = np.zeros(env.action_space.n)
            
            N[state][action] += 1
            V[state][action] = (total_returns[state][action] / N[state][action])
        state = next_state

            
#save the value function
fname = 'Results/Chapter '+str(chapter)+'/Level '+str(level)+'/MC_'+('every' if EVERY_VISIT_MC else 'first')+'_'+str(EPISODES)+'_episodes.bin'
with open(fname, 'wb') as handle:
    pickle.dump(V, handle, protocol=pickle.HIGHEST_PROTOCOL)
    

  0%|          | 0/1000 [00:00<?, ?it/s]

100%|██████████| 1000/1000 [00:31<00:00, 32.22it/s]


In [5]:
N.values()

dict_values([array([268., 158., 166., 222., 186.]), array([144.,  86.,  90., 121.,  97.]), array([44., 29., 27., 40., 36.]), array([25., 15., 13., 18., 17.]), array([ 5.,  5.,  2.,  3., 10.]), array([15., 11., 12., 17., 15.]), array([27., 16., 15., 23., 20.]), array([26., 13.,  8., 20., 15.]), array([17., 10., 15., 16., 15.]), array([20., 13., 11., 17., 15.]), array([19., 10., 13., 14., 13.]), array([123.,  75.,  75., 104.,  85.]), array([43., 26., 27., 33., 30.]), array([55., 35., 36., 48., 38.]), array([44., 27., 28., 31., 19.]), array([32., 29., 27., 42., 35.]), array([61., 38., 38., 54., 45.]), array([23., 13., 15., 21., 18.]), array([13., 10.,  9., 11., 10.]), array([9., 0., 6., 7., 4.]), array([18., 11., 11., 15., 13.]), array([30., 16., 19., 23., 19.]), array([13.,  7.,  8.,  9., 10.]), array([21., 15., 15., 23., 19.]), array([44., 25., 28., 37., 24.]), array([24., 15., 12., 20., 17.]), array([10.,  8.,  7.,  0., 14.]), array([1., 0., 6., 2., 8.]), array([13., 15., 13., 14.,  9.

In [6]:
V

{'3333333333333300000000000037717708004053000000000000333333333333330000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000': array([19.43306716, 19.48288608, 19.44839759, 19.48413964, 19.76744086]),
 '333333333333330000000000003770170800405300000000000033333333333333000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000

In [7]:

for s in V:
    argmax = np.argmax(V[s])
    max_val = np.max(V[s])
    s = s[26:40] +'\n'+ s[26+26:40+26]
    print("{} Action: {} Value: {}".format(s,ACTION_LOOKUP[argmax],max_val))

37717708004053
33333333333333 Action: extinguish Value: 19.767440860215
37701708004053
33333333333333 Action: extinguish Value: 37.86544329896897
37710708004053
33333333333333 Action: right Value: 117.71477272727262
37100708004053
33333333333333 Action: down Value: 215.96007692307694
31000708004053
33333333333333 Action: down Value: 1399.1385000000002
31000708004053
33333333333333 Action: left Value: 255.40758823529353
31000708004053
33333333333333 Action: up Value: 187.95387500000038
30100708004053
33333333333333 Action: right Value: 187.2019615384612
31000708004053
33333333333333 Action: left Value: 271.2616874999994
30100708004053
33333333333333 Action: right Value: 254.1932999999996
30010708004053
33333333333333 Action: left Value: 310.1010714285708
37107708004053
33333333333333 Action: extinguish Value: 43.23531764705871
31007708004053
33333333333333 Action: up Value: 117.55800000000028
31007708004053
33333333333333 Action: extinguish Value: 95.36792105263135
31007708004053
333333

In [8]:
def greedy_policy_pi(V,s):
    #.9 prob of greedy action
    #.1 prob of random action

    if s not in V:
        V[s] = np.zeros(env.action_space.n)
    r_choice = .3
    if np.random.random() < r_choice:
        return np.random.choice(np.arange(env.action_space.n))
    else:
        max_val = np.max(V[s])
         #find all actions that have the max value and choose one at random
        max_actions = np.argwhere(V[s] == max_val).flatten()
        return np.random.choice(max_actions)


In [3]:
_ = env.reset()
state = env.unwrapped.serialize_state()


In [10]:

for i in range(100):
    time.sleep(1)
    _ = env.reset()
    state = env.unwrapped.serialize_state()
    st_time = time.time()
    done = False
    for t in range(200):
        if time.time() - st_time < .2:
            env.render()
            time.sleep(1/60)
            continue
        st_time = time.time()
        action_time = time.time()
        action = greedy_policy_pi(V,state)
        next_state, reward, done, info = env.step(action)
        next_state = env.unwrapped.serialize_state()
        print("\rAction: {} Reward: {}".format(ACTION_LOOKUP[action],reward),end="")
        if done:
            continue
        env.render()

333300333
333000333
333000333
333000333
333000333
333000333
333000333
333000333
333000333
333000333
333000333
333003333
333300333
333000333
333000333
333000333
333000333
333000333
333000333
333000333
333000333
333000333
333000333
333003333
Action: extinguish Reward: 1.1399999999999992

KeyboardInterrupt: 

In [11]:
ACTION_LOOKUP

{0: 'right', 1: 'up', 2: 'down', 3: 'left', 4: 'extinguish'}

In [17]:

action = 4
next_state, reward, done, info = env.step(action)
env.render()
print("Action: {} Reward: {}".format(ACTION_LOOKUP[action],reward))

Action: extinguish Reward: 2.3920000000000026
